In [2]:
import numpy as np
import pandas as pd
import scipy

In [3]:
dados = scipy.io.loadmat(r"C:\Users\jmlnn\Downloads\barra_6kv_condi_1.mat")
print(dados.keys())

dict_keys(['__header__', '__version__', '__globals__', 'absoluto'])


In [4]:
dados.values()

dict_values([b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Fri Oct 25 09:54:30 2024', '1.0', [], array([[ 1.26266406e-01,  1.07125781e-01,  9.69695313e-02, ...,
         1.34078906e-01,  1.22164844e-01,  1.38571094e-01],
       [ 2.42780885e+01,  4.71812595e+01,  7.35990319e+01, ...,
         2.06046990e+02,  2.27669605e+02,  2.49815282e+02],
       [ 1.77637807e-02,  2.64073337e-02,  2.51448184e-02, ...,
         1.96421184e-02,  7.16884847e-03,  1.33674614e-02],
       ...,
       [ 1.77637807e-02,  1.18216758e-02,  2.71589820e-02, ...,
         1.23586275e-02,  8.76761038e-03,  2.09166100e-03],
       [ 1.93106094e-02, -9.40774910e-04,  7.01734600e-03, ...,
         3.61843832e-03, -8.24961128e-04,  6.32008617e-03],
       [ 1.77637807e-02,  4.52884681e-03,  1.70881640e-02, ...,
        -3.66505262e-03,  1.35638961e-02,  1.61864116e-02]])])

In [5]:
df = pd.DataFrame(list(dados.values())[-1])
df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.126266,0.107126,0.096970,0.111032,0.145798,0.116891,0.103610,0.098727,0.150290,0.132712,...,0.133102,0.108688,0.097165,0.099509,0.119235,0.105173,0.118454,0.134079,0.122165,0.138571
1,24.278088,47.181260,73.599032,203.379397,223.703278,248.527455,32.626977,52.742221,214.536815,236.797458,...,265.587097,20.008590,40.848259,55.651693,28.259999,48.991628,76.367047,206.046990,227.669605,249.815282
2,0.017764,0.026407,0.025145,0.009647,0.016724,0.022530,0.017878,0.028654,0.009726,0.015429,...,0.031525,0.018840,0.037155,0.008801,0.017173,0.019469,0.002447,0.019642,0.007169,0.013367
3,0.025498,0.019115,0.023131,0.027237,0.011365,0.014176,0.014108,0.014806,0.009726,0.009543,...,0.005112,0.018840,0.015044,0.020578,-0.000845,0.015755,0.009042,0.022556,0.021558,0.016186
4,0.013123,0.017291,0.025145,0.011406,0.018063,0.000809,0.025418,0.008871,0.012325,0.008071,...,0.006580,0.011652,0.004993,0.012727,0.022087,0.010184,0.022233,0.015272,0.010366,0.013367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,0.022404,0.020938,0.009032,0.021960,0.007346,0.019188,0.014108,0.036567,0.012325,0.009543,...,0.021254,-0.006318,0.025094,0.014689,-0.004121,0.019469,0.023882,0.010902,0.015163,0.013367
399,0.005389,0.013645,-0.001039,0.018442,0.006007,0.012505,0.015993,0.024697,0.001929,0.013958,...,0.005112,0.008058,0.013034,0.012727,0.007345,0.021327,0.018935,0.022556,0.011965,0.013367
400,0.017764,0.011822,0.027159,0.013165,0.015384,0.000809,0.021648,0.026675,0.004528,0.011014,...,0.006580,-0.002724,0.013034,0.012727,0.018811,0.006470,0.013989,0.012359,0.008768,0.002092
401,0.019311,-0.000941,0.007017,0.004370,0.016724,0.020859,0.012223,0.008871,0.011026,0.005127,...,0.021254,0.015246,0.013034,0.014689,0.013897,0.010184,0.010691,0.003618,-0.000825,0.006320


In [6]:
# Criar a coluna de classe
df.loc['classe'] = ['positivo' if valor <= 180 else 'negativo' for valor in df.iloc[1, :]]
df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,0.126266,0.107126,0.09697,0.111032,0.145798,0.116891,0.10361,0.098727,0.15029,0.132712,...,0.133102,0.108688,0.097165,0.099509,0.119235,0.105173,0.118454,0.134079,0.122165,0.138571
1,24.278088,47.18126,73.599032,203.379397,223.703278,248.527455,32.626977,52.742221,214.536815,236.797458,...,265.587097,20.00859,40.848259,55.651693,28.259999,48.991628,76.367047,206.04699,227.669605,249.815282
2,0.017764,0.026407,0.025145,0.009647,0.016724,0.02253,0.017878,0.028654,0.009726,0.015429,...,0.031525,0.01884,0.037155,0.008801,0.017173,0.019469,0.002447,0.019642,0.007169,0.013367
3,0.025498,0.019115,0.023131,0.027237,0.011365,0.014176,0.014108,0.014806,0.009726,0.009543,...,0.005112,0.01884,0.015044,0.020578,-0.000845,0.015755,0.009042,0.022556,0.021558,0.016186
4,0.013123,0.017291,0.025145,0.011406,0.018063,0.000809,0.025418,0.008871,0.012325,0.008071,...,0.00658,0.011652,0.004993,0.012727,0.022087,0.010184,0.022233,0.015272,0.010366,0.013367
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399,0.005389,0.013645,-0.001039,0.018442,0.006007,0.012505,0.015993,0.024697,0.001929,0.013958,...,0.005112,0.008058,0.013034,0.012727,0.007345,0.021327,0.018935,0.022556,0.011965,0.013367
400,0.017764,0.011822,0.027159,0.013165,0.015384,0.000809,0.021648,0.026675,0.004528,0.011014,...,0.00658,-0.002724,0.013034,0.012727,0.018811,0.00647,0.013989,0.012359,0.008768,0.002092
401,0.019311,-0.000941,0.007017,0.00437,0.016724,0.020859,0.012223,0.008871,0.011026,0.005127,...,0.021254,0.015246,0.013034,0.014689,0.013897,0.010184,0.010691,0.003618,-0.000825,0.00632
402,0.017764,0.004529,0.017088,0.030755,0.012705,0.019188,0.008453,0.022719,0.013625,0.009543,...,0.010982,0.013449,0.033135,0.014689,0.007345,0.026898,0.015638,-0.003665,0.013564,0.016186


In [30]:
# Filtrar os elementos do semi-ciclo positivo
dados_sp = df.loc[:, df.loc['classe'] == 'positivo'].drop('classe').to_numpy()[2:, :]
dados_sp = dados_sp.astype(float)  # Garantir que todos os elementos sejam floats

In [31]:
# Filtrar elementos do semi-ciclo negativo
dados_sn = df.loc[:, df.loc['classe'] == 'negativo'].drop('classe').to_numpy()[2:, :]
dados_sn = dados_sn.astype(float)  # Garantir que todos os elementos sejam floats

In [9]:
matrix = np.array(list(dados.values())[-1])
pulses = matrix[2:, :]
pulses

array([[ 0.01776378,  0.02640733,  0.02514482, ...,  0.01964212,
         0.00716885,  0.01336746],
       [ 0.02549792,  0.0191145 ,  0.02313065, ...,  0.02255551,
         0.02155771,  0.01618641],
       [ 0.01312329,  0.0172913 ,  0.02514482, ...,  0.01527202,
         0.01036637,  0.01336746],
       ...,
       [ 0.01776378,  0.01182168,  0.02715898, ...,  0.01235863,
         0.00876761,  0.00209166],
       [ 0.01931061, -0.00094077,  0.00701735, ...,  0.00361844,
        -0.00082496,  0.00632009],
       [ 0.01776378,  0.00452885,  0.01708816, ..., -0.00366505,
         0.0135639 ,  0.01618641]])

In [28]:
class SignalMetrics():

    def __init__(self, pulses: np.ndarray):
        """
        Inicializa a classe com um array de pulsos.

        Args:
            pulses(ndarray): Array NumPy contendo valores dos pulsos.
        """

        if not isinstance(pulses, np.ndarray): # Validações de entrada
            raise TypeError("Pulses devem ser um array NumPy.")
        if pulses.size == 0:
            raise ValueError("Pulses não pode ser vazio.")
        
        self.pulses = pulses
        self._peak_value = None # prefixo _ indica que são variáveis internas (convenção Python)
        self._rms = None
        self._avg_amplitude = None

    @property # permite acessar o método como se fosse um atributo (sem parênteses)
    def peak_value(self) -> float:
        """Calcula e retorna o valor de pico do sinal"""
        if self._peak_value is None:
            self._peak_value = np.max(np.abs(self.pulses), axis=1)
        return self._peak_value
    
    @property
    def rms(self) -> float:
        """Calcula e retorna o valor RMS do sinal"""
        if self._rms is None:
            self._rms = np.sqrt(np.mean(self.pulses**2, axis=1))
        return self._rms
    
    @property
    def avg_amplitude(self) -> float:
        """Calcula e retorna a amplitude média da matriz de pulsos"""
        if self._avg_amplitude is None:
            self._avg_amplitude = np.mean(np.abs(self.pulses))
        return self._avg_amplitude
    
    @property
    def sqrt_amplitude(self) -> float:
        """Calcula e retorna a raiz quadrada da amplitude média da matriz"""
        return np.sqrt(self.avg_amplitude)

    def get_crest_factor(self) -> float:
        """Calcula e retorna o fator de crista da matriz"""
        return self.peak_value / self.rms
    
    def get_clearance_factor(self) -> float:
        """Calcula e retorna o fator de liberação da matriz"""
        return self.peak_value / self.sqrt_amplitude
    
    def impulse_factor(self) -> float:
        """Calcula e retorna o fator de impulso da matriz"""
        return self.peak_value / self.avg_amplitude
    
    def shape_factor(self) -> float:
        """Calcula e retorna o fator de forma da matriz"""
        return self.rms / self.avg_amplitude
    
    def skewness(self) -> float:
        """Calcula e retorna a assimetria da matriz"""
        return scipy.stats.skew(self.pulses, axis=1)    

    def kurtosis(self) -> float:
        """Calcula e retorna a curtose da matriz"""
        return scipy.stats.kurtosis(self.pulses, axis=1)

In [29]:
metrics = SignalMetrics(dados_sp)
print(metrics)

metrics.rms

array([0.01976437, 0.01991753, 0.01822112, 0.01708636, 0.01881336,
       0.01515609, 0.01860446, 0.01863543, 0.0221633 , 0.01923404,
       0.01571615, 0.01611036, 0.01723918, 0.01728375, 0.02051364,
       0.01791548, 0.017836  , 0.01361408, 0.01721991, 0.01848216,
       0.0198137 , 0.02041393, 0.01800867, 0.01799996, 0.01706872,
       0.01655925, 0.01702903, 0.01909167, 0.01938186, 0.01797029,
       0.01862281, 0.02073419, 0.01730364, 0.01821216, 0.0167458 ,
       0.01868501, 0.02002701, 0.0180355 , 0.01800288, 0.01750776,
       0.01745949, 0.01769549, 0.01731099, 0.01846811, 0.02182714,
       0.01566549, 0.01607756, 0.01806169, 0.01736885, 0.01849006,
       0.01835932, 0.01965504, 0.01729854, 0.0156238 , 0.01817301,
       0.0191691 , 0.02020319, 0.0188845 , 0.01643288, 0.017797  ,
       0.01589345, 0.01834765, 0.01810172, 0.01704174, 0.01798811,
       0.01707927, 0.01604455, 0.01916275, 0.01719383, 0.01869125,
       0.01762217, 0.01813697, 0.0163791 , 0.01837831, 0.01953